In [1]:
pdb_file = "./pdbfiles/P3HT.pdb"
mol2file = './mol2files/MD/P3HT.mol2'
xml_filename = "./xml_files/NEW.xml"
typed_mol2 = "./mol2files/P3HT_typed.mol2"

In [2]:
from functions import esp_functions
from functions.esp_functions import openff_Molecule, espaloma, dictionaries, typing, esp_to_xml

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
LICENSE: No product keys!


# To Parameterize a molecule all you need to do is import the functions and run the following line:

In [3]:
esp_to_xml(MOL2FILEPATH=mol2file,XMLFILEPATH=xml_filename,PDBFILEPATH=pdb_file,TYPEDFILEPATH=typed_mol2)

RadicalsNotSupportedError: The OpenFF Toolkit does not currently support parsing molecules with S- and P-block radicals. Found 1 radical electrons on molecule [H][C]1S[C]([C]2S[C]([H])[C](C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])[C]2[H])[C](C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])[C]1[H].[H][C]1S[C]([H])[C](C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])[C]1[H].

## Children Issue 

at this point we've successfully parameterized our monomer and dimer and resaved them with the correct atom types to match the forcefield file. We just have to determine how to extract the monomer so we can build polymers from it. We have to build the polymers from the typed monomer so it'll have the correct atom types to be able to apply the forcefield. 

In [ ]:
p3ht_typed = mb.load(typed_mol2)
p3ht_typed.visualize()

try to access the mbuild children to pull just the monomer out to use for polymer building:

the p3ht_typed file should have two children (the dimer and the monomer), however it only has one. 
We either have to figure out how to rmeove the dimer from this mbuild compound or extract the monomer 
to use for polymer building 

In [ ]:
len(p3ht_typed.children)

Theres not an easy way to determine which atoms are part of the monomer and which are part of the dimer, but maybe if we could figure out where the origin lies we could determine that all the dimers coordinates are negative, for instance, and delete all the atoms with negative coordinates?

When we save the dimer and monomer together (done in the polymer builder ipynb) we could move the dimer into negative coordinates and the monomer into positive?

In [ ]:
#I saved the dimer in negative coordinates and monomer in positive
#This tells me that indices 0-10 and 33-48 belong to the monomer and the rest belong to the dimer 

for i in range(p3ht_typed.n_particles):
    print(i,p3ht_typed[i].name,p3ht_typed[i].xyz)

This is where I'm looking for ideas:
1) either pull the monomer out and create a new mbuild compound of the monomer
2) delete the dimer coordinates 
3) only parameterize the dimer, then remove the dimer bond and create a monomer from the dimer by deteting half of the dimer

In [ ]:
#this doesn't quite work the way I was hoping 

dimer_indices = []
for i in range(p3ht_typed.n_particles):
    if p3ht_typed[i].xyz[0][0] < 1:
        dimer_indices.append(i)
        
for i in dimer_indices:
    p3ht_typed.remove(p3ht_typed[i])